# Coupon Redemption: Category Prediction

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
from pathlib import Path

## Load Data

In [32]:
cwd = Path(os.getcwd())
os.path.join(str(cwd.parent.absolute()), '/data/Predicting Coupon Redemption/train')

'/data/Predicting Coupon Redemption/train'

In [28]:
cwd = Path(os.getcwd())
data_path = 
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
demo_df = pd.read_csv(os.path.join(data_path, 'customer_demographics.csv'))
cmpn_df = pd.read_csv(os.path.join(data_path, 'campaign_data.csv'))
cp_it_df = pd.read_csv(os.path.join(data_path, 'coupon_item_mapping.csv'))
txn_df = pd.read_csv(os.path.join(data_path, 'customer_transaction_data.csv'))
item_df = pd.read_csv(os.path.join(data_path, 'item_data.csv'))

FileNotFoundError: [Errno 2] No such file or directory: '/data/Predicting Coupon Redemption/train/train.csv'

In [3]:
txn_df.shape

(1324566, 7)

In [4]:
txn_df = txn_df.merge(item_df[['item_id', 'category']], on='item_id', how='left')
txn_df.shape

(1324566, 8)

In [5]:
txn_df['txn_month'] = txn_df['date'].str[:7]

In [6]:
txn_df.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,category,txn_month
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,Natural Products,2012-01
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,Natural Products,2012-01
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,Grocery,2012-01
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,Grocery,2012-01
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,Grocery,2012-01


## Clean Data

In [7]:
# drop duplicate rows
print('duplicated rows: ', txn_df.duplicated().sum())
print('drop duplicate rows')
txn_df.drop_duplicates(inplace=True)
print('duplicated rows: ', txn_df.duplicated().sum())

duplicated rows:  2916
drop duplicate rows
duplicated rows:  0


In [8]:
# remove space and symbols from category name
categories = txn_df['category'].unique()
new_categories = [re.sub('\s+', "", re.sub("[^A-Za-z0-9_ ]", "", cat_nm)) for cat_nm in categories]
cat_nm_map = dict(zip(categories, new_categories))
txn_df['category'] = txn_df['category'].map(cat_nm_map)
txn_df['category'].value_counts(dropna=False)

Grocery              939995
Pharmaceutical       154847
PackagedMeat          65981
NaturalProducts       52902
Meat                  23049
DairyJuicesSnacks     20435
Bakery                18196
Fuel                  13792
PreparedFood          13045
Seafood                7026
SkinHairCare           4856
Miscellaneous          3190
FlowersPlants          1601
Alcohol                1341
Salads                  428
Garden                  413
Travel                  246
Restauarant             211
Vegetablescut            96
Name: category, dtype: int64

In [9]:
all_ids = txn_df['customer_id'].unique()
print(f'there are total {len(all_ids)} unique user ids')

there are total 1582 unique user ids


## Prep Label

In [10]:
def prep_label(df, tnx_months=None):
    label = df.pivot_table(index=['customer_id', 'txn_month'], columns=['category'], values=['quantity'], aggfunc='count')
    flat_cols = [col[1] for col in label.columns]
    label.columns = flat_cols
    label.reset_index(inplace=True)
    if tnx_months is None:
        tnx_months = label['txn_month'].unique()
    print('total event months:', tnx_months)
    
    for m in tnx_months:
        m_user_ids = set(label[label['txn_month']==m]['customer_id'].unique())
        nopurchase_ids = set(all_ids).difference(m_user_ids)
        print(f'add {len(nopurchase_ids)} users with no purchase to label df for month {m}')
        
        nopurchase_label = pd.DataFrame(nopurchase_ids, columns=['customer_id'])
        nopurchase_label['txn_month'] = m
        for col in label.columns[2:]:
            nopurchase_label[col] = np.nan
        label = pd.concat([label, nopurchase_label], ignore_index=True)
    return label

In [11]:
label_df = prep_label(txn_df)

total event months: ['2012-02' '2012-03' '2012-04' '2012-05' '2012-06' '2012-07' '2012-08'
 '2012-09' '2012-10' '2012-11' '2012-12' '2013-01' '2013-02' '2013-03'
 '2013-04' '2013-05' '2013-06' '2012-01' '2013-07']
add 928 users with no purchase to label df for month 2012-02
add 532 users with no purchase to label df for month 2012-03
add 119 users with no purchase to label df for month 2012-04
add 138 users with no purchase to label df for month 2012-05
add 128 users with no purchase to label df for month 2012-06
add 117 users with no purchase to label df for month 2012-07
add 106 users with no purchase to label df for month 2012-08
add 93 users with no purchase to label df for month 2012-09
add 102 users with no purchase to label df for month 2012-10
add 99 users with no purchase to label df for month 2012-11
add 100 users with no purchase to label df for month 2012-12
add 95 users with no purchase to label df for month 2013-01
add 85 users with no purchase to label df for month 2013-

## Check Positive Ratio

In [12]:
class_cols = label_df.columns[2:]

In [13]:
for each_class in class_cols:
    row = label_df[each_class].notnull()
    label_df.loc[row, each_class] = 1
label_df.fillna(0, inplace=True)

In [14]:
label_df.head()

,customer_id,txn_month,Alcohol,Bakery,DairyJuicesSnacks,FlowersPlants,Fuel,Garden,Grocery,Meat,...,NaturalProducts,PackagedMeat,Pharmaceutical,PreparedFood,Restauarant,Salads,Seafood,SkinHairCare,Travel,Vegetablescut
0,1,2012-02,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2012-03,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2012-04,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2012-05,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2012-06,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
label_df.to_parquet(os.path.join(data_path, 'label.parquet'))

ArrowInvalid: Cannot parse URI: '../data/Predicting Coupon Redemption/train/label.parquet'

In [19]:
from pathlib import Path
path = Path(os.getcwd())

/Users/sukhumarn.a/Documents/Data Science/RL for Marketing/marketing_rl
